In [1]:
import cv2
import mediapipe as mp
import math

# Calculate distance between two points
def euclidean_distance(p1, p2):
    return math.dist(p1, p2)

# Eye Aspect Ratio (basic version)
def is_eye_open(eye_landmarks):
    # Upper and lower lid midpoints
    vertical = euclidean_distance(eye_landmarks[1], eye_landmarks[5])
    horizontal = euclidean_distance(eye_landmarks[0], eye_landmarks[3])
    ratio = vertical / horizontal
    return ratio > 0.25  # Threshold (tweak based on tests)


In [2]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
                                  max_num_faces=1,
                                  refine_landmarks=True,
                                  min_detection_confidence=0.5,
                                  min_tracking_confidence=0.5)

# Eye landmark indices from MediaPipe (right and left)
RIGHT_EYE = [33, 160, 158, 133, 153, 144]
LEFT_EYE  = [362, 385, 387, 263, 373, 380]


I0000 00:00:1745617640.714099   40808 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1745617640.716305   40889 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.3), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


W0000 00:00:1745617640.719894   40870 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745617640.739604   40870 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [3]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Flip for natural webcam feel
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape
            right_eye = [(int(face_landmarks.landmark[i].x * w),
                          int(face_landmarks.landmark[i].y * h)) for i in RIGHT_EYE]
            left_eye = [(int(face_landmarks.landmark[i].x * w),
                         int(face_landmarks.landmark[i].y * h)) for i in LEFT_EYE]

            right_open = is_eye_open(right_eye)
            left_open = is_eye_open(left_eye)

            eye_status = "Open" if right_open and left_open else "Closed"
            cv2.putText(frame, f"Eyes: {eye_status}", (30, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if eye_status == "Open" else (0, 0, 255), 2)

    cv2.imshow("Eye Detection", frame)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


W0000 00:00:1745617644.051016   40873 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


KeyboardInterrupt: 